## Collecting Yelp Reviews

In [143]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re

In [145]:
headers = headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
url="https://www.yelp.ca/biz/meet-on-main-vancouver?sort_by=date_desc"
response = requests.get(url, headers=headers)

In [3]:
content = response.content
soup = BeautifulSoup(content,"lxml")

In [5]:
reviews = json.loads(soup.find('script', type='application/ld+json').string)['review']
reviews[0]

{'author': 'Tyson L.',
 'datePublished': '2021-06-26',
 'reviewRating': {'ratingValue': 5},
 'description': 'Rarely will you find MeeT empty -- an arbitrary indicator of popularity if you care about that -- as the vegan food is of high quality with solid portions. \n\nAfter multiple visits, some with Vegans, others with the &quot;oh but you need animal protein to survive cohort&quot;, 90% of guests would recommend the restaurant for its all-inclusive, varying menu.\n\nMy test for a good restaurant is whether I could replicate the dishes and/or be willing to create the dishes, and I could not with the same flavors.\n\nPersonally a big fan of the Macro bowl or the Meet burger, but most friends are content with the selection of poutines and the Mighty Mac.\n\nGood quality food is good quality food, and whether you&apos;re vegan or not you will be satisfied'}

In [19]:
sample_rev = reviews[0]['description']
sample_rev

'Rarely will you find MeeT empty -- an arbitrary indicator of popularity if you care about that -- as the vegan food is of high quality with solid portions. \n\nAfter multiple visits, some with Vegans, others with the &quot;oh but you need animal protein to survive cohort&quot;, 90% of guests would recommend the restaurant for its all-inclusive, varying menu.\n\nMy test for a good restaurant is whether I could replicate the dishes and/or be willing to create the dishes, and I could not with the same flavors.\n\nPersonally a big fan of the Macro bowl or the Meet burger, but most friends are content with the selection of poutines and the Mighty Mac.\n\nGood quality food is good quality food, and whether you&apos;re vegan or not you will be satisfied'

In [144]:
regex = r'&.{4};'
unwanted = set()
for i in re.finditer(regex, sample_rev):
    unwanted.add(i.group())
    
punc = '''!()-[]{};:'"\,<>./?@#$^&*_~'''

In [106]:
def clean_sent(text):
    cleaned = []
    for word in text.split():
        if not word.isalpha():
            for unwanted_string in unwanted:
                if unwanted_string in word:
                    word = word.replace(unwanted_string, "")
            word = word.strip(punc)
            cleaned.append(word)
        else:
            cleaned.append(word)
    
    return " ".join(cleaned)

In [107]:
clean_sent(sample_rev)

'Rarely will you find MeeT empty  an arbitrary indicator of popularity if you care about that  as the vegan food is of high quality with solid portions After multiple visits some with Vegans others with the oh but you need animal protein to survive cohort 90% of guests would recommend the restaurant for its all-inclusive varying menu My test for a good restaurant is whether I could replicate the dishes and/or be willing to create the dishes and I could not with the same flavors Personally a big fan of the Macro bowl or the Meet burger but most friends are content with the selection of poutines and the Mighty Mac Good quality food is good quality food and whether youre vegan or not you will be satisfied'

In [137]:
def get_rating(rating_dict):
    return rating_dict['ratingValue']

In [138]:
yelp_df = pd.DataFrame.from_dict(reviews)
yelp_df['reviewRating'] = yelp_df['reviewRating'].apply(get_rating)
yelp_df['description'] = yelp_df['description'].apply(clean_sent)
yelp_df

author datePublished  reviewRating  \
0      Tyson L.    2021-06-26             5   
1    Melissa E.    2021-10-09             4   
2       Matt B.    2021-07-30             4   
3       Jill N.    2021-03-18             5   
4     Maling S.    2020-09-17             5   
5       Dana S.    2021-02-18             5   
6   Michelle H.    2020-05-08             4   
7    Jasmine Y.    2021-09-15             5   
8      Aaron W.    2021-05-24             1   
9   Jennifer B.    2020-03-12             5   
10    Purple Z.    2021-08-10             5   
11     Niall O.    2021-09-29             3   
12     Chloe C.    2020-09-11             5   
13      John R.    2021-02-02             3   
14    Norman S.    2020-02-25             4   
15      Elsa C.    2020-01-31             3   
16      Chef D.    2020-09-24             4   
17   Patrick F.    2021-01-27             5   
18    Autumn M.    2021-04-08             5   
19        A. W.    2021-07-31             2   

                                          description  
0   Rarely will you find MeeT empty  an arbitrary ...  
1   Im not a vegan but I love trying innovative ve...  
2   Im much more sold on this place nowadays I ord...  
3   Went here for St Patties Day Our server Derek ...  
4   Sept 17 and went with my brother for vegan foo...  
5   Ive ordered takeout from the Yaletown location...  
6   I havent had this is YEARS I used to be SO obs...  
7   Ordered takeout online to the Main Street loca...  
8   Saw 3 patio tables empty and barely any line u...  
9   Love the MeeT restaurants in Vancouver Ive eat...  
10  Ive been a couple times to the restaurant in G...  
11  The staff all seemed really nice and their was...  
12  I probably wont be dining out as often since s...  
13  I used love MeeT on Main I order MeeT via door...  
14  Never thought vegetarian food could taste this...  
15  It was a rainy Friday in January A group of 8 ...  
16  This spot always serves up the yummiest vegan ...  
17  A great selection of Vegan food The sweet chil...  
18  Its taken me way too long to review this place...  
19  Why is the service here so poor The food is ok...

In [141]:
yelp_df.to_csv('yelp_revs.csv', index=False)